In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage , HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import sqlite3
import uuid

load_dotenv()
google = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# Create State
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    

def chat_node(state: ChatState):
    messages = state['messages']
    response = google.invoke(messages)
    return {"messages": [response]}

In [2]:
def generate_thread_id():
    thread_id = uuid.uuid4()
    return thread_id

In [3]:
# adding database
conn = sqlite3.connect(database='testing.db' , check_same_thread=False)
checkpointer = SqliteSaver(conn=conn)

In [4]:

graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)
chatbot = graph.compile(checkpointer=checkpointer)

In [5]:

CONFIG = {'configurable': {'thread_id':"newthread"}}


In [6]:
result = chatbot.invoke(
                {'messages': [HumanMessage(content="thatnks for the help")]},
                config=CONFIG,
)
result['messages']


[HumanMessage(content='Why is my Name', additional_kwargs={}, response_metadata={}, id='1a639b1d-374a-47f0-b869-a43c21ffa68a'),
 AIMessage(content="Your name is what your parents or guardians chose to give you.  There's no single reason why they chose that specific name; it could be for any number of reasons, including:\n\n* **Family tradition:**  The name might be a family name passed down through generations.\n* **Meaning and significance:**  They might have liked the meaning or symbolism of the name.\n* **Sound and style:** They might have simply liked the sound or style of the name.\n* **Personal connection:** The name might remind them of someone they admire or love.\n* **Cultural influences:**  The name might be common or significant within their cultural background.\n* **Popularity:**  It might have been a popular name at the time of your birth.\n\nTo know *why* your parents chose your name, you'd need to ask them.", additional_kwargs={}, response_metadata={'prompt_feedback': {'

In [7]:
for x in checkpointer.list(None):
    print(x.config['configurable']['thread_id']) 
    break

newthread


In [11]:
messages = chatbot.get_state(config={'configurable': {'thread_id': 'newthread'}}).values['messages']

In [ ]:
for msg in messages:
    print(msg.content)

content='Why is my Name' additional_kwargs={} response_metadata={} id='1a639b1d-374a-47f0-b869-a43c21ffa68a'
content="Your name is what your parents or guardians chose to give you.  There's no single reason why they chose that specific name; it could be for any number of reasons, including:\n\n* **Family tradition:**  The name might be a family name passed down through generations.\n* **Meaning and significance:**  They might have liked the meaning or symbolism of the name.\n* **Sound and style:** They might have simply liked the sound or style of the name.\n* **Personal connection:** The name might remind them of someone they admire or love.\n* **Cultural influences:**  The name might be common or significant within their cultural background.\n* **Popularity:**  It might have been a popular name at the time of your birth.\n\nTo know *why* your parents chose your name, you'd need to ask them." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_rating

In [23]:
for msg in messages:
    if isinstance(msg, HumanMessage):
        print("user : " , msg.content)
    else:
        print("bot : " , msg.content)

user :  Why is my Name
bot :  Your name is what your parents or guardians chose to give you.  There's no single reason why they chose that specific name; it could be for any number of reasons, including:

* **Family tradition:**  The name might be a family name passed down through generations.
* **Meaning and significance:**  They might have liked the meaning or symbolism of the name.
* **Sound and style:** They might have simply liked the sound or style of the name.
* **Personal connection:** The name might remind them of someone they admire or love.
* **Cultural influences:**  The name might be common or significant within their cultural background.
* **Popularity:**  It might have been a popular name at the time of your birth.

To know *why* your parents chose your name, you'd need to ask them.
user :  What is my Name
bot :  I do not know your name.  I have no access to personal information about you unless you explicitly provide it to me.
user :  my name is pranay
bot :  It's nice 